In [1]:
from atfunc import *
import pandas as pd
import os
import datetime
import numpy as np

def save_var(month):
    f = open("variables.py","w") 
    L = [
        "# Step 1 output:\n",
        "output_pivot_table = 'Output/"+month+'/PivotTable_'+month+".xlsx' \n",
        "output_parcel_cost = 'Output/"+month+'/Parcel_cost_'+month+".xlsx' \n",
        "# Step 2 (optimizer) output:\n",
        "optimizer_output = 'Output/"+month+"/optimizer_output_"+month+".xlsx' \n",
        "sheet_BRS = 'BRS cost Daywise' \n",
        "sheet_VU = 'Vehicle Utilisation Daywise' \n"
        "# Files required as input:\n",
        "filename_master = 'Master.xlsx' \n",
        "sheet_cost = 'BRS' \n",
        "sheet_truck_cap = 'Truck' \n",
        "filename_summary = 'Summary/"+month+".xlsx' \n",
        "sheet_summary = 'Summary Sheet' \n",
        ]

    f.writelines(L) 
    f.close()
    

month = input('Enter month: ')
save_var(month)

Enter month: December


In [2]:
parcel_sites = set()
Psheet = pd.read_excel('Master.xlsx','Parcel').values.ravel()
[parcel_sites.add(x) for x in Psheet];

parcel_locations = set()
site = pd.read_excel('Master.xlsx','Site').values
[parcel_locations.add(r[3].split(' ')[0].upper()) for r in site if r[0] in parcel_sites];

del_sheet = pd.read_excel('Master.xlsx','Delete').values
[parcel_locations.add(r[1].split(' ')[0].upper()) for r in del_sheet if r[0] in parcel_sites];

In [3]:
Master_SKU = pd.read_excel("Master.xlsx", 'Master SKU')
cm = Master_SKU.columns.tolist()
Master_SKU = Master_SKU[[cm[0],cm[2],cm[9],cm[10]]]
cm = Master_SKU.columns.tolist()

In [4]:
stn_path = 'STNs/'+month+'/' 
STN = pd.DataFrame([])
STN_parcel = pd.DataFrame([])
files = os.listdir(stn_path)

for f in files:
    
    stn = pd.read_excel(stn_path+f)
    c = stn.columns.tolist()
    stn = stn[[c[0],c[2],c[6],c[7],c[9],c[12]]]
    
    t=stn[['TOLOCATION']].values.ravel().tolist()
    t = [x.upper() for x in t]
    df = pd.DataFrame(t)
    stn[['TOLOCATION']]=df 
    
    if stn['TOLOCATION'][0].split(' ')[0] not in parcel_locations:
        
        stn = pd.merge(stn, Master_SKU[[cm[1],cm[2],cm[3]]],left_on='Product Name',right_on='Item_Des',how = 'left')
        stn = pd.merge(stn, Master_SKU[[cm[0],cm[2],cm[3]]],left_on='Product Code',right_on='Mas_Pcode', how = 'left')
        stn['Weight'] = np.where(stn['Weight_x'].isna(), stn['Weight_y'], stn['Weight_x'])
        stn['Volume'] = np.where(stn['Volume_x'].isna(), stn['Volume_y'], stn['Volume_x'])
           
        stn = stn.dropna(subset=['Weight', 'Volume'])
        stn = stn.assign(Weight_kg = stn.Weight * stn.Case)
        stn = stn.assign(Volume_m3 = stn.Volume * stn.Case)

        c = stn.columns.tolist()
        stn = stn[[c[0],c[1],c[5],c[14],c[15]]]
        
        if len(STN.values.tolist())==0:
            STN = stn
        else:
            STN = STN.append(stn,ignore_index=True)
    else:
        
        stn = stn[['TOLOCATION','LocTfrDate','Case']]
        if len(STN_parcel.values.tolist())==0:
            STN_parcel = stn
        else:
            STN_parcel = STN_parcel.append(stn, ignore_index=True)
        
        
make_pivot_table(STN, month)
make_parcel_cost_table(STN_parcel, month)    


Making pivot table
Pivot table created 
Making pivot table for parcel sites
case

0       68.583333
1       85.458333
2      136.791670
3       21.708333
4       65.208333
5        1.000000
6        1.000000
7        1.000000
8        1.000000
9        1.000000
10       2.000000
11       2.000000
12       1.000000
13       1.000000
14       2.000000
15       2.000000
16       1.000000
17       2.000000
18       1.000000
19       1.000000
20       1.000000
21       1.000000
22       2.000000
23       1.000000
24      10.000000
25       1.000000
26       2.000000
27       1.000000
28       1.000000
29       1.000000
          ...    
394   -136.791670
395    -85.458333
396    -68.583333
397    -21.708333
398      0.062500
399      1.000000
400      0.166667
401      0.187500
402      0.187500
403      1.000000
404      0.125000
405      0.312500
406      0.166667
407      0.250000
408      0.250000
409      0.083333
410      0.083333
411      0.300000
412      0.083333
413      0.250000


In [6]:
STN_parcel

TOLOCATION LocTfrDate        Case
0    DUMMY WAREHOUSE 2019-12-02   68.583333
1    DUMMY WAREHOUSE 2019-12-02   85.458333
2    DUMMY WAREHOUSE 2019-12-02  136.791670
3    DUMMY WAREHOUSE 2019-12-02   21.708333
4    DUMMY WAREHOUSE 2019-12-02   65.208333
5              DEWAS 2019-12-02    1.000000
6              NAGDA 2019-12-02    1.000000
7             UJJAIN 2019-12-02    1.000000
8            NEEMUCH 2019-12-02    1.000000
9          PITHAMPUR 2019-12-02    1.000000
10           NEEMUCH 2019-12-02    2.000000
11            RATLAM 2019-12-02    2.000000
12            UJJAIN 2019-12-02    1.000000
13             DEWAS 2019-12-02    1.000000
14              DHAR 2019-12-02    2.000000
15            BADWAH 2019-12-02    2.000000
16              DHAR 2019-12-02    1.000000
17          KHARGONE 2019-12-02    2.000000
18             NAGDA 2019-12-02    1.000000
19           NEEMUCH 2019-12-02    1.000000
20            UJJAIN 2019-12-02    1.000000
21              DHAR 2019-12-02    1.000000
22           NEEMUCH 2019-12-02    2.000000
23            UJJAIN 2019-12-02    1.000000
24             DEWAS 2019-12-02   10.000000
25             DEWAS 2019-12-02    1.000000
26              DHAR 2019-12-02    2.000000
27          KHARGONE 2019-12-02    1.000000
28          KHARGONE 2019-12-02    1.000000
29         PITHAMPUR 2019-12-02    1.000000
..               ...        ...         ...
394  DUMMY WAREHOUSE 2019-12-02 -136.791670
395  DUMMY WAREHOUSE 2019-12-02  -85.458333
396  DUMMY WAREHOUSE 2019-12-02  -68.583333
397  DUMMY WAREHOUSE 2019-12-02  -21.708333
398           UJJAIN 2019-12-03    0.062500
399         KHARGONE 2019-12-03    1.000000
400         KHARGONE 2019-12-03    0.166667
401         KHARGONE 2019-12-03    0.187500
402          BADWANI 2019-12-03    0.187500
403           JHABUA 2019-12-03    1.000000
404           JHABUA 2019-12-03    0.125000
405           RATLAM 2019-12-03    0.312500
406           RATLAM 2019-12-03    0.166667
407         KHARGONE 2019-12-03    0.250000
408         KHARGONE 2019-12-03    0.250000
409          NEEMUCH 2019-12-03    0.083333
410          NEEMUCH 2019-12-03    0.083333
411           RATLAM 2019-12-03    0.300000
412           RATLAM 2019-12-03    0.083333
413           RATLAM 2019-12-03    0.250000
414         KHARGONE 2019-12-03    0.040000
415         KHARGONE 2019-12-03    0.166667
416         KHARGONE 2019-12-03    0.100000
417           UJJAIN 2019-12-03    0.166667
418           UJJAIN 2019-12-03    0.208333
419         KHARGONE 2019-12-03    0.200000
420         KHARGONE 2019-12-03    0.166667
421         KHARGONE 2019-12-03    0.125000
422         KHARGONE 2019-12-03    0.500000
423           UJJAIN 2019-12-03    0.041667

[424 rows x 3 columns]

In [78]:
os.system('python optimizer.py')
os.system('python summary.py')


0

In [45]:
stn = pd.read_excel('o1.xlsx',headers=None)
s=stn[[4.6832]].values
s=s[0][0]

a = float('nan')
s==np.NaN


False

In [8]:
L = [
        "month = '"+month+"' \n",
        "filename_master = 'Master.xlsx' \n",
        "sheet_Parcel = 'Parcel' \n",
        "sheet_Parcel_Rates = 'Parcel Rates' \n",
        "stn_path = 'STNs/"+month+"/' \n",
        "sheet_merge_branches = 'milk run' \n",
        "sheet_delete_branches = 'Delete' \n",
        "output_pivot_table = 'Output/"+month+"/PivotTable_December.xlsx' \n",
        "output_parcel_cost = 'Output/"+month+"/Parcel_cost_December.xlsx' \n",
        "sheet_cost = 'BRS' \n",
        "sheet_truck_cap = 'Truck' \n",
        "optimizer_output = 'Output/"+month+"/optimizer_output_December.xlsx' \n",
        "filename_summary = 'Summary/"+month+".xlsx' \n",
        "sheet_BRS = 'BRS cost Daywise' \n",
        "sheet_summary = 'Summary Sheet' \n",
        "sheet_VU = 'Vehicle Utilisation Daywise' \n"
        ]  
